In [1]:
from rdkit import Chem
import numpy as np

def read_smiles(file_path):
    with open(file_path, 'r') as file:
        smiles = file.read().strip().split('\n')
    return smiles

def normalize_smiles(smiles_list):
    normalized = [Chem.MolToSmiles(Chem.MolFromSmiles(smile), canonical=True) for smile in smiles_list if Chem.MolFromSmiles(smile) is not None]
    return normalized

def create_char_tokenizer(smiles_list):
    all_chars = set(''.join(smiles_list))
    char_to_idx = {char: idx + 1 for idx, char in enumerate(sorted(all_chars))}
    idx_to_char = {idx: char for char, idx in char_to_idx.items()}
    return char_to_idx, idx_to_char

def tokenize_and_pad(smiles_list, char_to_idx):
    max_length = max(len(smile) for smile in smiles_list)
    tokenized = [[char_to_idx[char] for char in smile] for smile in smiles_list]
    padded_smiles = [s + [0] * (max_length - len(s)) for s in tokenized]
    return padded_smiles, max_length

file_path = 'smiles_train.txt'
smiles = read_smiles(file_path)
normalized_smiles = normalize_smiles(smiles)
char_to_idx, idx_to_char = create_char_tokenizer(normalized_smiles)
padded_smiles, max_length = tokenize_and_pad(normalized_smiles, char_to_idx)

print(f"Total number of molecules: {len(smiles)}")
print(f"Number of valid molecules after normalization: {len(normalized_smiles)}")
print(f"Max length of SMILES: {max_length}")
print(f"Vocabulary size: {len(char_to_idx)}")  # Vocabulary size based on unique characters

print("Example of normalized SMILES:", normalized_smiles[0])
print("Example of tokenized and padded SMILES:", padded_smiles[0])


Total number of molecules: 1036643
Number of valid molecules after normalization: 1036643
Max length of SMILES: 101
Vocabulary size: 37
Example of normalized SMILES: COc1ccc(N2CCN(C(=O)c3cc4ccccc4[nH]3)CC2)cc1
Example of tokenized and padded SMILES: [19, 23, 29, 8, 29, 29, 29, 3, 22, 9, 19, 19, 22, 3, 19, 3, 17, 23, 4, 29, 10, 29, 29, 11, 29, 29, 29, 29, 29, 11, 26, 33, 20, 27, 10, 4, 19, 19, 9, 4, 29, 29, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

class SMILESDataset(Dataset):
    def __init__(self, smiles, labels):
        self.smiles = smiles
        self.labels = labels

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, idx):
        return torch.tensor(self.smiles[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

In [9]:
class SMILESModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(SMILESModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embeddings = self.embedding(x)
        lstm_out, _ = self.lstm(embeddings)
        logits = self.fc(lstm_out)
        return logits

In [10]:
vocab_size = len(char_to_idx) + 1
embedding_dim = 64
hidden_dim = 256
num_layers = 2
batch_size = 32


input_seqs = [smile[:-1] for smile in padded_smiles]
target_seqs = [smile[1:] for smile in padded_smiles]

train_inputs, train_targets = input_seqs[:int(len(input_seqs) * 0.8)], target_seqs[:int(len(input_seqs) * 0.8)]
val_inputs, val_targets = input_seqs[int(len(input_seqs) * 0.8):], target_seqs[int(len(input_seqs) * 0.8):]

train_dataset = SMILESDataset(train_inputs, train_targets)
val_dataset = SMILESDataset(val_inputs, val_targets)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [7]:
from tqdm.notebook import tqdm 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SMILESModel(vocab_size, embedding_dim, hidden_dim, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

import os
model_save_path = 'model_checkpoints'
os.makedirs(model_save_path, exist_ok=True)

epochs = 12
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", mininterval=1.0)
    for inputs, labels in train_pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.transpose(1, 2), labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_pbar.set_postfix({'loss': loss.item()})

    model.eval()
    valid_loss = 0
    val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Validate]", mininterval=1.0)
    with torch.no_grad():
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.transpose(1, 2), labels)
            valid_loss += loss.item()
            val_pbar.set_postfix({'val_loss': loss.item()})

    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = valid_loss / len(val_loader)
    print(f"Average Training Loss: {avg_train_loss}, Average Validation Loss: {avg_val_loss}")

    if (epoch + 1) % 2 == 0:
        checkpoint_filename = f"checkpoint_epoch_{epoch+1}.pth"
        checkpoint_filepath = os.path.join(model_save_path, checkpoint_filename)
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': valid_loss,
        }, checkpoint_filepath)
        print(f"Checkpoint saved to {checkpoint_filepath}")


Epoch 1/12 [Train]:   0%|          | 0/25917 [00:00<?, ?it/s]

Epoch 1/12 [Validate]:   0%|          | 0/6480 [00:00<?, ?it/s]

Average Training Loss: 0.32578375104224144, Average Validation Loss: 0.29426086135898477


Epoch 2/12 [Train]:   0%|          | 0/25917 [00:00<?, ?it/s]

Epoch 2/12 [Validate]:   0%|          | 0/6480 [00:00<?, ?it/s]

...

Epoch 11/12 [Validate]:   0%|          | 0/6480 [00:00<?, ?it/s]

Average Training Loss: 0.2638689709376897, Average Validation Loss: 0.26812668766274494


Epoch 12/12 [Train]:   0%|          | 0/25917 [00:00<?, ?it/s]

Epoch 12/12 [Validate]:   0%|          | 0/6480 [00:00<?, ?it/s]

Average Training Loss: 0.26357746301986795, Average Validation Loss: 0.2680020926060316
Checkpoint saved to model_checkpoints\checkpoint_epoch_12.pth


In [11]:
import torch
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SMILESModel(vocab_size, embedding_dim, hidden_dim, num_layers).to(device)
model_save_path = 'model_checkpoints'

checkpoint_filename = "checkpoint_epoch_12.pth"
checkpoint_filepath = os.path.join(model_save_path, checkpoint_filename)

checkpoint = torch.load(checkpoint_filepath)
model.load_state_dict(checkpoint['model_state_dict'])

print("Model loaded from checkpoint")


Model loaded from checkpoint


In [93]:
import torch
import torch.nn.functional as F

def generate_smiles(model, char_to_idx, idx_to_char, seed='C', max_length=100):
    model.eval()
    device = next(model.parameters()).device
    
    current_sequence = seed
    input_sequence = [char_to_idx[char] for char in current_sequence]
    input_tensor = torch.tensor([input_sequence], dtype=torch.long).to(device)
    
    for _ in range(max_length):
        with torch.no_grad():
            outputs = model(input_tensor)
            probabilities = F.softmax(outputs[:, -1, :], dim=-1).squeeze()
            predicted_idx = torch.multinomial(probabilities, 1).item()

            if predicted_idx not in idx_to_char:
                break 

            predicted_char = idx_to_char[predicted_idx]

            if predicted_char == '\n':
                break

            current_sequence += predicted_char
            input_sequence.append(predicted_idx)
            input_tensor = torch.tensor([input_sequence], dtype=torch.long).to(device)

    return current_sequence


def generate_large_number_of_smiles(model, num_smiles, seed_initial='C'):
    generated_smiles = []
    for _ in tqdm(range(num_smiles), desc="Generating SMILES"):
        smiles = generate_smiles(model, char_to_idx, idx_to_char, seed=seed_initial, max_length=100)
        generated_smiles.append(smiles)
    return generated_smiles

num_generation = 11200
all_generated_smiles = generate_large_number_of_smiles(model, num_generation)


Generating SMILES:   0%|          | 0/11200 [00:00<?, ?it/s]

In [94]:
from rdkit import Chem
from rdkit.Chem import QED
from tqdm.notebook import tqdm
from rdkit import RDLogger

def validate_and_score_smiles(smiles_list):
    RDLogger.DisableLog('rdApp.*')
    valid_smiles_scores = []
    failed_smiles = []

    for smiles in tqdm(smiles_list, desc="Validating and Scoring SMILES"):
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                Chem.Kekulize(mol, clearAromaticFlags=True)
                qed_score = QED.qed(mol)
                valid_smiles_scores.append((smiles, qed_score))
        except (Chem.KekulizeException, ValueError) as e:
            print(f"Failed to process SMILES {smiles}: {str(e)}")
            failed_smiles.append(smiles)

    return valid_smiles_scores, failed_smiles

validated_smiles, failed_smiles = validate_and_score_smiles(all_generated_smiles)


Validating and Scoring SMILES:   0%|          | 0/11200 [00:00<?, ?it/s]

In [95]:
def save_top_smiles(validated_smiles, num_top_smiles, filename='top_generated_smiles.txt'):

    sorted_smiles = sorted(validated_smiles, key=lambda x: x[1], reverse=True)
    top_smiles = sorted_smiles[:num_top_smiles]
    
    with open(filename, 'w') as file:
        for smiles, score in top_smiles:
            file.write(f"{smiles}\n")

save_top_smiles(validated_smiles, 10000)


In [96]:
import pandas as pd
from resources.utils import canonicalize_smiles, get_Pareto_fronts

def load_smiles(filename):
    return pd.read_csv(filename, header=None, names=['smiles'])['smiles'].tolist()

smiles_real = load_smiles("smiles_train.txt")
smiles_gen = load_smiles("top_generated_smiles.txt")

sample_size = 10000
smiles_real_sample = np.random.choice(smiles_real, sample_size, replace=False)
smiles_gen_sample = np.random.choice(smiles_gen, sample_size, replace=False)

smiles_real_can = [w for w in canonicalize_smiles(smiles_real_sample) if w is not None]
smiles_gen_can = [w for w in canonicalize_smiles(smiles_gen_sample) if w is not None]

In [97]:
validity = len(smiles_gen_can) / len(smiles_gen)
print("Validity: ", validity)

Validity:  1.0


In [98]:
smiles_unique = set(smiles_gen_can)
uniqueness = len(smiles_unique) / len(smiles_gen)
print("Uniqueness: ", uniqueness)

Uniqueness:  0.9995


In [99]:
smiles_novel = set(smiles_gen_can) - set(smiles_real_can)
novelty = len(smiles_novel) / len(smiles_gen)
print("Novelty:", novelty)

Novelty: 0.9993


In [100]:
import fcd
ref_model = fcd.load_ref_model()

In [101]:
# Get CHEMBLNET activations of generated molecules 
act_real = fcd.get_predictions(ref_model, smiles_real_can)
act_gen = fcd.get_predictions(ref_model, smiles_gen_can)

# Calculate mean and covariance statistics from these activations for both sets
mu_real = np.mean(act_real, axis=0)
sigma_real = np.cov(act_real.T)

mu_gen = np.mean(act_gen, axis=0)
sigma_gen = np.cov(act_gen.T)

# Calculate the FCD
fcd_value = fcd.calculate_frechet_distance(
    mu1=mu_real,
    mu2=mu_gen, 
    sigma1=sigma_real,
    sigma2=sigma_gen)

print('FCD: ', fcd_value)

FCD:  1.069638603612944
